In [ ]:
import os
import glob
import json
import tqdm
import zipfile
import io, json, re, zipfile, pickle, html
from pathlib import Path
from typing import Any, Dict, List, Optional
import pandas as pd
from config.settings import Settings 
from utils import *
from json_extractor import JsonExtractor
settings = Settings()

# Extract content geel into dataframe

In [13]:
extracted = {}
json_docs = get_all_json_docs(content_type="geel")
for key, doc in json_docs.items():
    extracted_data = extract_json(key,doc)
    extracted[key] = extracted_data
extracted_df = pd.DataFrame(extracted.values())
print(f"Extracted {len(extracted_df)} documents")

Extracted 6146 documents


In [14]:
extracted_df_nodupes = extracted_df.drop_duplicates(subset='title')
print(f"Removed {len(extracted_df) - len(extracted_df_nodupes)} duplicates")

Removed 773 duplicates


### Koppelling met KM nummer

In [18]:
km_title_map = pd.read_excel(settings.content_folder / 'titel_km_map.xlsx').rename(columns={'Content Title':'title',"Article Identifier": 'km_nummer'})
km_title_map = km_title_map.drop_duplicates('title').set_index('title')
extracted_df_km= extracted_df_nodupes.set_index('title').join(km_title_map).dropna(subset='km_nummer')

In [21]:
extracted_df_km.to_pickle(settings.content_folder / "extracted_df_km.pkl")

### Lijst met duplicaten exporteren

In [10]:
extracted_df.set_index("id").loc[extracted_df.set_index("id").index.difference(extracted_df_nodupes.set_index("id").index)].to_excel("dupelist.xlsx")

In [1]:
import json

In [6]:
json.loads("""{
  "content": "## OB - Waarom ontvang ik een aangiftebrief?\n\nU ontvangt de aangiftebrief omdat wij u als ondernemer willen herinneren om op tijd aangifte omzetbelasting (OB) te doen. In de brief staan per aangiftetijdvak de uiterste aangiftedatum, de uiterste betaaldatum en het betalingskenmerk voor het lopende jaar.\n\nDe verzendperiodes zijn afhankelijk van uw aangiftetijdvak:\n\n- Doet u maandaangifte, u ontvangt de aangiftebrief in januari.\n- Doet u kwartaalaangifte, u ontvangt de aangiftebrief in maart.\n- Doet u jaaraangifte, u ontvangt de aangiftebrief in december.\n\nVoor startende ondernemers, of bij een wijziging van het aangiftetijdvak of van de aangifteplicht, versturen wij de ‘aangiftebrief Omzetbelasting’ in dezelfde week als het klaarzetten van de eerste digitale aangifte na de startdatum of wijziging.\n\nWij zetten de aangifte klaar op Mijn Belastingdienst Zakelijk (MBD-Z). Als u een e-mailadres hebt opgegeven op MBD-Z, ontvangt u ook een e-mail wanneer de aangifte klaarstaat.\n\nAls u uw aangifte al heeft gedaan, kunt u de aangiftebrief als niet verzonden beschouwen. Wij sturen de brief ook als extra service om u aan te herinneren tijdig aangifte te doen.\n\nAls wij de aangifte niet meer digitaal kunnen klaarzetten voor het tijdvak waarvoor u aangifte moet doen, sturen we binnen 10 werkdagen na de brief 'Vaststelling belastingplicht' een startersaangifte toe.\n\n\nPrivéantwoord (voor medewerkers)\n\nActie: controleren waarom de ondernemer een aangiftebrief ontving\n- Controleer op MBD-Z of de aangifte digitaal klaarstaat.\n- Controleer in EBVR (taak 3240) de ingangsdatum of wijzigingsdatum van de aangifteplicht.\n- Als de aangifte niet klaarstaat: kijk of er een startersaangifte is verzonden.\n- Raadpleeg DAS op berichtsjabloon OB140 met extensie AFP om te zien of en welke startersaangifte(n) zijn verzonden. Bij meerdere jaren: scroll in het sjabloon naar beneden om de verzonden aangiftes te zien.\n\nSysteemverwijzing: waar informatie staat\n- Mijn Belastingdienst Zakelijk (MBD-Z): hier zetten we aangiftes klaar en hier ziet u of de aangifte klaarstaat.\n- EBVR (taak 3240): voor ingangs- en wijzigingsdatum van de aangifteplicht.\n- DAS, berichtsjabloon OB140 (extensie AFP): controle of startersaangifte verzonden is.\n\nHandelingsperspectief: wat te doen afhankelijk van situatie\n- Is de ondernemer startend of is er een wijziging in tijdvak/ aangifteplicht? Controleer of de aangifte in dezelfde week is klaargezet als de eerste digitale aangifte na de start/wijziging.\n- Heeft de ondernemer een e-mailadres in MBD-Z? Controleer of er een e-mail is verzonden dat de aangifte klaarstaat.\n- Kunt u de aangifte niet digitaal klaarzetten voor het betreffende tijdvak? Controleer of binnen 10 werkdagen na de brief 'Vaststelling belastingplicht' een startersaangifte is toegezonden.\n- Informeer de ondernemer over de Btw-Alert app als extra service voor herinneringen.\n\nBronvermelding interne kennis: KM1006352, KM1013862, KM1011980"
}""")

JSONDecodeError: Invalid control character at: line 2 column 59 (char 60)